In [1]:
import os
os.chdir('../')
os.getcwd()

'e:\\github_clone\\GridFormer-14'

In [2]:
from gridformer.vq_vae import VQVAE
from gridformer.trainer import VQTrainer
from gridformer.Utils.utils import Config
from gridformer.Utils.dataset import load_observation, ObsDataset, DataLoader
import torch
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import torch.nn as nn
import lightsim2grid

In [3]:
config = Config.from_yaml('config.yml')
model = VQVAE(config=config)
trainer = VQTrainer(config=config, model=model)

In [4]:
obs = load_observation("C:\\Users\\Ernest\\Downloads\\topo_data", start=0, end=5)
dataset = ObsDataset(obs, device=model.device)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
data_variance = np.var(obs)

In [8]:
trainer.train_data(epochs=10, data_variance=data_variance, dataloader=dataloader)

Epoch 1 - Reconstruction Loss: 0.1079, 
Model saved at 0.108
Epoch 2 - Reconstruction Loss: 0.1027, 
Model saved at 0.103
Epoch 3 - Reconstruction Loss: 0.0866, 
Model saved at 0.087
Epoch 4 - Reconstruction Loss: 0.0879, 
Epoch 5 - Reconstruction Loss: 0.1068, 
Epoch 6 - Reconstruction Loss: 0.0788, 
Model saved at 0.079
Epoch 7 - Reconstruction Loss: 0.1074, 
Epoch 8 - Reconstruction Loss: 0.0928, 
Epoch 9 - Reconstruction Loss: 0.0744, 
Model saved at 0.074
Epoch 10 - Reconstruction Loss: 0.1070, 
Epoch 11 - Reconstruction Loss: 0.0943, 
Epoch 12 - Reconstruction Loss: 0.0935, 
Epoch 13 - Reconstruction Loss: 0.0900, 
Epoch 14 - Reconstruction Loss: 0.0988, 
Epoch 15 - Reconstruction Loss: 0.0988, 
Epoch 16 - Reconstruction Loss: 0.0978, 
Epoch 17 - Reconstruction Loss: 0.0861, 
Epoch 18 - Reconstruction Loss: 0.1068, 
Epoch 19 - Reconstruction Loss: 0.1105, 
Epoch 20 - Reconstruction Loss: 0.0960, 
Epoch 21 - Reconstruction Loss: 0.0795, 
Epoch 22 - Reconstruction Loss: 0.1002, 
Ep

In [9]:
model.load_model()

Model loaded successfully from gridformer\models\encoder
Model loaded successfully from gridformer\models\decoder
Model loaded successfully from gridformer\models\vq


In [10]:
obs_t = torch.tensor(np.array(obs[100], dtype=np.float32), dtype=torch.float32, device=model.device)

In [11]:
pred_x, vq_loss, encodings = model(obs_t.unsqueeze(0))

In [12]:
F.mse_loss(pred_x, obs_t) / data_variance

C:\Users\Ernest\AppData\Local\Temp\ipykernel_11232\2610654153.py:1: UserWarning: Using a target size (torch.Size([467])) that is different to the input size (torch.Size([1, 467])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  F.mse_loss(pred_x, obs_t) / data_variance


tensor(0.1320, device='cuda:0', grad_fn=<DivBackward0>)

In [33]:
z_q = model.vq.embedding(latent_indices.to(model.device))  # Maps indices back to codebook vectors

# Step 3: Pass z_q to the decoder
generated_data = model.decoder(z_q)

In [40]:
latent_indices[0]

tensor([ 67,  84, 103,  81,   4,  11,  31,  78,   5, 103, 118,  21,  91,  17,
         68, 126,  93,  75,  57,  55,  68, 122,  36, 110,  91, 100,  37,  22,
        108,  96,  67,  32,  96,  50,  68,  43,  19,  24,  59,   8,  82, 117,
         28,  21,  62,  71,  76, 109,  59,  11,  24,   9,  53,  17,  59,  46,
         71,  83, 112,  16,  16, 102,  22,  62])

In [10]:
class VectorQuantizer(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, commitment_cost):
        super(VectorQuantizer, self).__init__()

        self._embedding_dim = embedding_dim
        self._num_embeddings = num_embeddings

        self._embedding = nn.Embedding(self._num_embeddings, self._embedding_dim)
        self._embedding.weight.data.uniform_(-1/self._num_embeddings, 1/self._num_embeddings)
        self._commitment_cost = commitment_cost

    def forward(self, inputs):
        # convert inputs from BCHW -> BHWC
        inputs = inputs.permute(0, 2, 3, 1).contiguous()
        input_shape = inputs.shape

        # Flatten input
        flat_input = inputs.view(-1, self._embedding_dim)
        print(flat_input.shape)

        # Calculate distances
        distances = (torch.sum(flat_input**2, dim=1, keepdim=True)
                    + torch.sum(self._embedding.weight**2, dim=1)
                    - 2 * torch.matmul(flat_input, self._embedding.weight.t()))

        # Encoding
        encoding_indices = torch.argmin(distances, dim=1).unsqueeze(1)
        print(encoding_indices.shape)
        encodings = torch.zeros(encoding_indices.shape[0], self._num_embeddings, device=inputs.device)
        encodings.scatter_(1, encoding_indices, 1)

        # Quantize and unflatten
        quantized = torch.matmul(encodings, self._embedding.weight)
        print(quantized.shape)
        quantized = quantized.view(input_shape)

        # Loss
        e_latent_loss = F.mse_loss(quantized.detach(), inputs) # commitment loss
        q_latent_loss = F.mse_loss(quantized, inputs.detach()) # detach stop gradient
        loss = q_latent_loss + self._commitment_cost * e_latent_loss

        quantized = inputs + (quantized - inputs).detach() # trick, 通过常数让编码器与解码器连续，可导
        avg_probs = torch.mean(encodings, dim=0)
        perplexity = torch.exp(-torch.sum(avg_probs * torch.log(avg_probs + 1e-10)))

        # convert quantized from BHWC -> BCHW
        return loss, quantized.permute(0, 3, 1, 2).contiguous(), perplexity, encodings

In [11]:
vq = VectorQuantizer(num_embeddings=512, embedding_dim=64, commitment_cost=0.25)

# Generate dummy image data (B, C, H, W) -> Batch, Channels, Height, Width
dummy_data = torch.randn(8, 3, 32, 32)  # Example: Batch of 8, 64 channels, 32x32 resolution

# Forward pass through the VectorQuantizer
loss, quantized, perplexity, encodings = vq(dummy_data)

# Print results
print("Quantization Loss:", loss.item())
print("Quantized Output Shape:", quantized.shape)  # Should match the input shape
print("Perplexity:", perplexity.item())
print("Encodings Shape:", encodings.shape)  # Should be (Ba

torch.Size([384, 64])
torch.Size([384, 1])
torch.Size([384, 64])
Quantization Loss: 1.2756234407424927
Quantized Output Shape: torch.Size([8, 3, 32, 32])
Perplexity: 245.60606384277344
Encodings Shape: torch.Size([384, 512])


In [7]:
encodings[0].shape

torch.Size([512])

In [13]:
import torch
import torch.nn as nn

# Modified VectorQuantizer for 1D data
class VectorQuantizer1D(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, commitment_cost):
        super(VectorQuantizer1D, self).__init__()
        self._embedding_dim = embedding_dim
        self._num_embeddings = num_embeddings
        self._embedding = nn.Embedding(self._num_embeddings, self._embedding_dim)
        self._embedding.weight.data.uniform_(-1 / self._num_embeddings, 1 / self._num_embeddings)
        self._commitment_cost = commitment_cost

    def forward(self, inputs):
        # Add a channel dimension to inputs: (B, C, L) -> (B, L, C)
        inputs = inputs.unsqueeze(-1)  # Shape (Batch, 467) -> (Batch, 467, 1)
        inputs = inputs.permute(0, 2, 1).contiguous()  # Shape (Batch, 467, 1) -> (Batch, 1, 467)
        input_shape = inputs.shape  # Save the shape for reshaping later
        print(input_shape)
        # Flatten input
        flat_input = inputs.view(-1, self._embedding_dim)
        print(flat_input.shape)
        # Calculate distances
        distances = (torch.sum(flat_input**2, dim=1, keepdim=True)
                     + torch.sum(self._embedding.weight**2, dim=1)
                     - 2 * torch.matmul(flat_input, self._embedding.weight.t()))
        
        # Encoding
        encoding_indices = torch.argmin(distances, dim=1).unsqueeze(1)
        encodings = torch.zeros(encoding_indices.shape[0], self._num_embeddings, device=inputs.device)
        encodings.scatter_(1, encoding_indices, 1)
        
        # Quantize and unflatten
        quantized = torch.matmul(encodings, self._embedding.weight).view(input_shape)
        
        # Loss
        e_latent_loss = torch.nn.functional.mse_loss(quantized.detach(), inputs)  # Commitment loss
        q_latent_loss = torch.nn.functional.mse_loss(quantized, inputs.detach())  # Detach gradient
        loss = q_latent_loss + self._commitment_cost * e_latent_loss
        
        # Trick for gradient flow
        quantized = inputs + (quantized - inputs).detach()
        avg_probs = torch.mean(encodings, dim=0)
        perplexity = torch.exp(-torch.sum(avg_probs * torch.log(avg_probs + 1e-10)))
        
        # Reshape quantized output back to (Batch, 467)
        quantized = quantized.permute(0, 2, 1).squeeze(-1)
        return loss, quantized, perplexity, encodings

# Example 1D data
num_embeddings = 128
embedding_dim = 1
commitment_cost = 0.25
vq1d = VectorQuantizer1D(num_embeddings=num_embeddings, embedding_dim=embedding_dim, commitment_cost=commitment_cost)

dummy_1d_data = torch.randn(10, 64)  # Batch size 10, Data length 467
loss, quantized, perplexity, encodings = vq1d(dummy_1d_data)

# Print results

print("Quantization Loss:", loss.item())
print("Quantized Output Shape:", quantized.shape)  # Should match input shape
print("Perplexity:", perplexity.item())
print("Encodings Shape:", encodings.shape)  # Should be (Batch * 467, num_embeddings)


torch.Size([10, 1, 64])
torch.Size([640, 1])
Quantization Loss: 1.2371002435684204
Quantized Output Shape: torch.Size([10, 64])
Perplexity: 2.0640199184417725
Encodings Shape: torch.Size([640, 128])


In [14]:
encodings[0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.])

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Assume the encoder output
latent_dim = 64  # Output dimension of the encoder
num_categories_per_chunk = 10  # Number of categories per chunk
num_chunks = 32  # Number of discrete categories

# Example latent space output from encoder (batch_size=1 for simplicity)
encoder_output = torch.randn(1, latent_dim)  # Shape: (1, 64)

# Step 1: Reshape the latent vector into chunks
chunk_size = latent_dim // num_chunks  # Size of each chunk (64/32 = 2 here)
chunks = encoder_output.view(-1, num_chunks, chunk_size)  # Shape: (batch_size, 32, 2)

# Step 2: Map each chunk to a categorical distribution
projection_layer = nn.Linear(chunk_size, num_categories_per_chunk)  # Map each chunk to logits
projected_logits = projection_layer(chunks)  # Shape: (batch_size, 32, num_categories_per_chunk)

# Step 3: Convert logits to probabilities using softmax
probabilities = F.softmax(projected_logits, dim=-1)  # Shape: (batch_size, 32, num_categories_per_chunk)

# Step 4: Sample discrete indices for each chunk
categorical_dist = torch.distributions.Categorical(probabilities)
sampled_indices = categorical_dist.sample()  # Shape: (batch_size, 32)

# Step 5: (Optional) Use argmax for deterministic indices
argmax_indices = torch.argmax(probabilities, dim=-1)  # Shape: (batch_size, 32)

# Print results
print("Encoder Output (latent space):", encoder_output)
print("Chunks:", chunks)
print("Projected Logits:", projected_logits)
print("Probabilities:", probabilities)
print("Sampled Indices (stochastic):", sampled_indices)
print("Argmax Indices (deterministic):", argmax_indices)


Encoder Output (latent space): tensor([[-0.3739,  0.9837,  0.4223,  1.6036, -0.7556,  0.8580,  0.0707,  0.5382,
          1.6785, -0.1444, -0.1950, -0.1330, -0.2951,  0.0589, -1.8625, -1.9954,
          1.0577,  0.7093, -0.3857, -0.9472, -0.9413, -0.5421, -2.2130,  0.0392,
         -0.5691,  1.6046,  0.5420,  1.0865, -1.3775, -0.1582, -1.3851, -0.7562,
         -0.0241,  0.8344, -2.2034,  0.0046,  0.6399,  0.8680, -1.2576, -0.8168,
          0.7748,  0.0445, -0.6579,  1.5881,  0.4967, -0.6689, -0.8986,  0.4288,
         -0.5296, -0.7484, -0.7317,  0.8956,  2.4192, -0.7451, -0.3974, -0.4048,
         -1.6695, -0.0093, -1.9778,  0.4404, -3.1932,  0.5183, -0.7743,  0.0718]])
Chunks: tensor([[[-0.3739,  0.9837],
         [ 0.4223,  1.6036],
         [-0.7556,  0.8580],
         [ 0.0707,  0.5382],
         [ 1.6785, -0.1444],
         [-0.1950, -0.1330],
         [-0.2951,  0.0589],
         [-1.8625, -1.9954],
         [ 1.0577,  0.7093],
         [-0.3857, -0.9472],
         [-0.9413, -0

In [1]:
import torch
import torch.nn as nn

# Example cookbook (num_embeddings, embedding_dim)
num_embeddings = 128
embedding_dim = 64
cookbook = nn.Embedding(num_embeddings, embedding_dim)
cookbook.weight.data.uniform_(-1/num_embeddings, 1/num_embeddings)  # Initialize embeddings

# Example encoder output (batch_size, embedding_dim)
batch_size = 10
encoder_output = torch.randn(batch_size, embedding_dim)

# Compute distances between encoder outputs and cookbook embeddings
# (batch_size, embedding_dim) -> (batch_size, num_embeddings)
distances = (
    torch.sum(encoder_output**2, dim=1, keepdim=True)  # (batch_size, 1)
    + torch.sum(cookbook.weight**2, dim=1)            # (num_embeddings)
    - 2 * torch.matmul(encoder_output, cookbook.weight.t())  # (batch_size, num_embeddings)
)

# Find the index of the closest embedding
indices = torch.argmin(distances, dim=1)  # (batch_size,)

print("Discrete Latent Space Indices:", indices)


Discrete Latent Space Indices: tensor([ 35,  31,  90,  32, 119,  87,  99, 117,  18, 116])


In [4]:
embedding_vector = cookbook(indices[0])

In [5]:
embedding_vector

tensor([ 2.9811e-03,  1.6618e-03,  2.1967e-03, -3.0517e-03, -1.6027e-03,
        -1.5963e-04, -4.2291e-03, -1.7923e-03,  3.7617e-03, -2.3688e-04,
        -6.2850e-03, -7.3592e-03, -2.9147e-03, -6.8549e-03,  2.2048e-03,
        -6.6810e-04,  2.4617e-03,  4.1050e-03,  5.8938e-03, -2.2384e-03,
        -5.0574e-03, -1.1509e-03,  9.5361e-04,  2.7614e-05,  1.7162e-03,
        -5.9978e-03,  4.0929e-03, -6.8525e-03,  7.5781e-03, -5.2806e-03,
        -4.5421e-03, -6.2159e-03,  6.3326e-03, -5.6456e-03, -4.6632e-03,
         6.1647e-03,  2.2271e-03,  1.3548e-03, -3.9303e-03,  2.4023e-03,
         2.2934e-04,  7.6142e-03, -4.8640e-03,  3.9350e-03,  7.3311e-03,
        -7.5137e-04,  5.4937e-03, -2.2966e-03,  9.7253e-04,  1.2131e-03,
        -2.6793e-03,  3.8246e-03,  7.4413e-03, -6.6107e-03,  2.5234e-03,
         7.0236e-03,  3.6490e-03,  2.2222e-03, -7.3964e-03, -5.1530e-03,
         5.4228e-03,  4.2103e-03,  6.4655e-03,  2.5434e-03],
       grad_fn=<EmbeddingBackward0>)

In [19]:
import torch
import torch.nn as nn

# Example cookbook
num_embeddings = 512
embedding_dim = 64
cookbook = nn.Embedding(num_embeddings, embedding_dim)
cookbook.weight.data.uniform_(-1 / num_embeddings, 1 / num_embeddings)

# Example encoder output
batch_size = 10
encoder_output = torch.randn(batch_size, embedding_dim)

# Compute distances for each data point to all embeddings
# Encoder output is (batch_size, embedding_dim)
# Cookbook weights are (num_embeddings, embedding_dim)

# Expand dimensions for broadcasting and compute distances
encoder_output_flat = encoder_output.unsqueeze(1)  # Shape: (batch_size, 1, embedding_dim)
cookbook_weight = cookbook.weight.unsqueeze(0)     # Shape: (1, num_embeddings, embedding_dim)

# Calculate squared distances
distances = torch.sum((encoder_output_flat - cookbook_weight) ** 2, dim=2)  # Shape: (batch_size, num_embeddings)

# Find the index of the closest embedding for each latent vector
indices = torch.argmin(distances, dim=1)  # Shape: (batch_size,)

# Retrieve discrete latent space from indices
discrete_latent_space = indices  # Shape: (batch_size,)
decoder_input = cookbook(discrete_latent_space)  # Convert indices to embeddings

# Print results
print("Discrete Latent Space Shape:", discrete_latent_space.shape)  # Shape: (10,)
print("Decoder Input Shape:", decoder_input.shape)  # Shape: (10, embedding_dim)


Discrete Latent Space Shape: torch.Size([10])
Decoder Input Shape: torch.Size([10, 64])


In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class VectorQuantizer(nn.Module):
    def __init__(self, config):
        """
        Vector Quantizer for 1D data with latent space compression.

        Args:
            config (object): Configuration object containing:
                - num_embeddings (int): Number of embedding vectors.
                - embedding_dim (int): Dimensionality of each embedding vector.
                - commitment_cost (float): Weight for commitment loss.
        """
        super(VectorQuantizer, self).__init__()
        self.config = config
        self._embedding_dim = self.config.embedding_dim
        self._num_embeddings = self.config.num_embeddings
        self._commitment_cost = self.config.commitment_cost

        # Embedding layer
        self._embedding = nn.Embedding(self._num_embeddings, self._embedding_dim)
        self._embedding.weight.data.uniform_(-1 / self._num_embeddings, 1 / self._num_embeddings)

    def forward(self, z):
        """
        Forward pass of the Vector Quantizer.

        Args:
            z (torch.Tensor): Latent representation from the encoder. Shape: (batch_size, embedding_dim)

        Returns:
            z_q (torch.Tensor): Quantized representation. Shape: (batch_size, embedding_dim)
            loss (torch.Tensor): VQ loss (reconstruction + commitment).
            indices (torch.Tensor): Indices of the nearest embeddings. Shape: (batch_size,)
        """
        # Compute distances to each embedding
        z_flattened = z.unsqueeze(1)  # Shape: (batch_size, 1, embedding_dim)
        embeddings = self._embedding.weight.unsqueeze(0)  # Shape: (1, num_embeddings, embedding_dim)
        distances = torch.sum((z_flattened - embeddings) ** 2, dim=2)  # Shape: (batch_size, num_embeddings)

        # Find nearest embeddings
        indices = torch.argmin(distances, dim=1)  # Shape: (batch_size,)

        # Retrieve quantized vectors
        z_q = self._embedding(indices)  # Shape: (batch_size, embedding_dim)

        # Compute VQ loss
        commitment_loss = self._commitment_cost * F.mse_loss(z.detach(), z_q)
        vq_loss = F.mse_loss(z_q, z.detach()) + commitment_loss

        # Use straight-through estimator for backpropagation
        z_q = z + (z_q - z).detach()

        return z_q, vq_loss, indices


In [21]:
# Example configuration
class Config:
    embedding_dim = 64
    num_embeddings = 512
    commitment_cost = 0.25

# Instantiate the model
config = Config()
vq_layer = VectorQuantizer(config)

# Input latent representation from encoder
batch_size = 10
latent_dim = config.embedding_dim
z = torch.randn(batch_size, latent_dim)

# Forward pass
z_q, vq_loss, indices = vq_layer(z)

# Print results
print("Quantized Output Shape:", z_q.shape)  # Should be (batch_size, embedding_dim)
print("VQ Loss:", vq_loss.item())
print("Indices Shape:", indices.shape)  # Should be (batch_size,)


Quantized Output Shape: torch.Size([10, 64])
VQ Loss: 1.3594753742218018
Indices Shape: torch.Size([10])


In [24]:
con = vq_layer._embedding(indices)

In [28]:
con[0]

tensor([ 3.0721e-04, -1.0879e-03, -1.5866e-03,  1.2390e-03,  4.6564e-04,
         1.2915e-03, -1.7815e-03, -1.7502e-03,  1.1934e-03,  1.4162e-03,
         1.1394e-03,  1.2946e-05,  1.3202e-03, -4.6986e-04, -1.8610e-03,
        -8.4364e-04,  5.0331e-04,  1.4153e-03,  7.8391e-04,  5.7348e-04,
         1.5097e-03,  7.4554e-04,  1.3382e-03,  5.1083e-04,  6.0783e-04,
        -1.6609e-03, -2.8362e-04, -5.3025e-04, -1.8239e-03,  5.5003e-04,
         1.2435e-03,  2.8530e-04,  1.5939e-03, -1.1185e-03, -1.4002e-03,
         1.9032e-03,  1.8885e-03,  9.7357e-04, -1.2733e-03, -2.3261e-04,
         1.4033e-03,  8.9949e-05, -5.5751e-04, -1.7965e-03, -6.8806e-04,
         1.0771e-04, -1.8458e-03,  1.1947e-04,  6.6283e-04,  5.0251e-04,
        -1.7891e-03, -1.2311e-03, -6.0251e-04, -1.2800e-03, -6.1387e-04,
         1.4836e-03, -1.3507e-03,  8.6210e-04, -6.8157e-04,  9.6605e-04,
        -1.4283e-03,  1.7177e-04, -1.5519e-04,  1.0003e-04],
       grad_fn=<SelectBackward0>)

In [29]:
z_q[0]

tensor([ 3.0721e-04, -1.0879e-03, -1.5866e-03,  1.2390e-03,  4.6563e-04,
         1.2915e-03, -1.7815e-03, -1.7502e-03,  1.1934e-03,  1.4162e-03,
         1.1394e-03,  1.2949e-05,  1.3202e-03, -4.6980e-04, -1.8611e-03,
        -8.4364e-04,  5.0330e-04,  1.4153e-03,  7.8391e-04,  5.7352e-04,
         1.5096e-03,  7.4553e-04,  1.3382e-03,  5.1093e-04,  6.0783e-04,
        -1.6608e-03, -2.8363e-04, -5.3024e-04, -1.8239e-03,  5.5002e-04,
         1.2435e-03,  2.8527e-04,  1.5939e-03, -1.1185e-03, -1.4002e-03,
         1.9032e-03,  1.8885e-03,  9.7358e-04, -1.2733e-03, -2.3261e-04,
         1.4032e-03,  8.9943e-05, -5.5754e-04, -1.7965e-03, -6.8808e-04,
         1.0771e-04, -1.8458e-03,  1.1945e-04,  6.6280e-04,  5.0247e-04,
        -1.7891e-03, -1.2311e-03, -6.0248e-04, -1.2800e-03, -6.1387e-04,
         1.4836e-03, -1.3507e-03,  8.6212e-04, -6.8152e-04,  9.6604e-04,
        -1.4283e-03,  1.7178e-04, -1.5521e-04,  1.0002e-04])